**Prerequisites:** This notebook requires additional packages for plotting and MCMC visualization. Install them with:
```
pip install matplotlib corner tqdm
```

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import corner
from VegasAfterglow import ObsData, Setups, Fitter, ParamDef, Scale

In [ ]:
######### 1. PREPARE DATA #########
#Create an instance of the ObsData class to store light curve and spectrum data
data = ObsData()

# Define the frequency bands [Hz] and corresponding light curve files
band = [2.4e17, 4.84e14, 1.4e14]
lc_files = ["data/ep.csv", "data/r.csv", "data/vt-r.csv"]

# Loop through each frequency band and corresponding file(s)
for nu, fname in zip(band, lc_files):
    # Read the light curve data from the CSV file (columns: t, Fv_obs, Fv_err)
    t_data, Fv_obs, Fv_err = np.loadtxt(fname, delimiter=',', skiprows=1, unpack=True)

    # Add the light curve data to the 'data' object for the given frequency band
    # You can assign weight to each data point for chi^2 evaluation
    # You don't need to worry about the weights' normalization, the code will normalize them automatically.
    data.add_flux_density(nu=nu, t=t_data, f_nu=Fv_obs, err=Fv_err, weights=np.ones(len(t_data)))  # All quantities in CGS units

# Define the times [s] for spectra and corresponding files
times = [3000]
spec_files = ["data/ep-spec.csv"]    

# Loop through each time and its corresponding spectrum file
for t, fname in zip(times, spec_files):
    # Read the spectrum data from the CSV file (columns: nu, Fv_obs, Fv_err)
    nu_data, Fv_obs, Fv_err = np.loadtxt(fname, delimiter=',', skiprows=1, unpack=True)

    # Add the spectrum data to the 'data' object for the given time
    data.add_spectrum(t=t, nu=nu_data,
                      f_nu=Fv_obs, err=Fv_err, weights=np.ones(len(nu_data)))  # All quantities in CGS units

######### 2. CONFIGURE MODEL #########
cfg = Setups()
cfg.lumi_dist = 3.364e28    # Luminosity distance [cm]  
cfg.z         = 1.58        # Redshift
cfg.medium    = "wind"      # Medium type: "wind", "ism" (Interstellar Medium)
cfg.jet       = "powerlaw"  # Jet structure: "powerlaw", "gaussian", "tophat", "two_component", "step_powerlaw" 

######### 3. DEFINE PARAMETERS #########
# Parameter name, lower bound, upper bound, scale type, initial value (optional)

mc_params = [
    ParamDef("E_iso",    1e51,  1e54,  Scale.LOG,    1e53),    # Isotropic energy [erg]
    ParamDef("Gamma0",      5,  1000,  Scale.LOG,      20),    # Lorentz factor at the core
    ParamDef("theta_c",   0.0,   0.5,  Scale.LINEAR,  0.3),    # Core half-opening angle [rad]
    ParamDef("k_e",         2,     2,  Scale.FIXED,     2),    # Energy power law index
    ParamDef("k_g",         2,     2,  Scale.FIXED,     2),    # Lorentz factor power law index
    ParamDef("theta_v",   0.0,   0.0,  Scale.FIXED,   0.0),    # Viewing angle [rad]
    ParamDef("p",           2,     3,  Scale.LINEAR,  2.3),    # Power law index
    ParamDef("eps_e",    1e-2,   0.3,  Scale.LOG,    0.05),    # Electron energy fraction
    ParamDef("eps_B",    1e-4,   0.3,  Scale.LOG,    0.03),    # Magnetic field energy fraction
    ParamDef("xi_e",     1e-3,   0.1,  Scale.LOG,    0.01),    # Electron acceleration efficiency
    ParamDef("A_star",   1e-3,    10,  Scale.LOG,    0.05),    # Wind parameter 
]

######### 4. RUN MCMC SAMPLING #########
# Create an instance of the Fitter class using the prepared data and configuration
fitter = Fitter(data, cfg)

# Run the fitting process using bilby with dynesty sampler
#result = fitter.fit(
#    mc_params,
#    resolution=(0.3, 0.3, 10), # Grid resolution (phi, theta, t) - affects model accuracy
#    sampler="dynesty",         # Use dynesty nested sampler
#    nlive=500,                 # Number of live points
#    dlogz=0.1,                 # Stopping criterion
#    top_k=10,                  # Number of best-fit parameters to return
#    outdir="bilby_output",     # Output directory for results
#    label="afterglow_fit_dynesty"    # Run label
#)

# Run the fitting process using bilby with emcee sampler
result = fitter.fit(
    mc_params,
    resolution=(0.2, 0.3, 10),# Grid resolution (phi, theta, t) - affects model accuracy
    sampler="emcee",          # Use emcee MCMC sampler 
    nsteps=10000,             # Number of steps per walker 
    nburn=2000,               # Burn-in steps to discard
    top_k=10,                 # Number of best-fit parameters to return
    outdir="bilby_output",    # Output directory for results#
    label="afterglow_fit_emcee"    # Run label##
)

print(f"Total samples: {result.samples.shape[0]}")

# Print top-k parameters
print("\nTop-k parameters:")
header = f"{'Rank':>4s}  {'chi^2':>10s}  " + "  ".join(f"{name:>10s}" for name in result.labels)
print(header)
print("-" * len(header))
for i in range(result.top_k_params.shape[0]):
    chi2 = -2 * result.top_k_log_probs[i]
    vals = "  ".join(f"{val:10.4f}" for val in result.top_k_params[i])
    print(f"{i+1:4d}  {chi2:10.2f}  {vals}")

In [ ]:
# Define time and frequency ranges for predictions
t_out = np.logspace(2, 9, 150)

nu_out = np.logspace(16,20,150)

best_params = result.top_k_params[0]

# Generate model light curves at the specified bands using the best-fit parameters
lc = fitter.flux_density_grid(best_params, t_out, band)

# Generate model spectra at the specified times using the best-fit parameters
spec = fitter.flux_density_grid(best_params, times, nu_out)

In [ ]:
# Function to plot model light curves along with observed data
def draw_bestfit(t,lc_fit, nu, spec_fit):
    fig =plt.figure(figsize=(4.5, 7.5))
    ax1 = fig.add_subplot(211)
    ax2 = fig.add_subplot(212)

    shift = [1,1,200]
    colors = ['blue', 'orange', 'green']
    for i, file, sft, c in zip(range(len(lc_files)), lc_files, shift, colors ):
        t_d, Fv_d, Fe_d = np.loadtxt(file, delimiter=',', skiprows=1, unpack=True)
        ax1.errorbar(t_d, Fv_d*sft, Fe_d*sft, fmt='o',markersize=4,label=file, color=c,markeredgecolor='k', markeredgewidth=.4)
        ax1.plot(t, np.array(lc_fit[i,:])*sft, color=c,lw=1)

    ax1.set_xscale('log')
    ax1.set_yscale('log')
    ax1.set_xlabel('t [s]')
    ax1.set_ylabel(r'$F_\nu$ [erg/cm$^2$/s/Hz]')
    ax1.legend()
  
    for i, file, sft, c in zip(range(len(spec_files)), spec_files, shift, colors ):
        nu_d, Fv_d, Fe_d = np.loadtxt(file, delimiter=',', skiprows=1, unpack=True)
        ax2.errorbar(nu_d, Fv_d*sft, Fe_d*sft, fmt='o',markersize=4,label=file, color=c,markeredgecolor='k', markeredgewidth=.4)
        ax2.plot(nu, np.array(spec_fit[:,i])*sft, color=c,lw=1)

    ax2.set_xscale('log')
    ax2.set_yscale('log')
    ax2.set_xlabel(r'$\nu$ [Hz]')
    ax2.set_ylabel(r'$F_\nu$ [erg/cm$^2$/s/Hz]')
    ax2.legend()
    plt.tight_layout()

draw_bestfit(t_out, lc, nu_out, spec)

In [ ]:
######### 5. VISUALIZATION #########

# Option 1: Use bilby's built-in corner plot (recommended - uses proper LaTeX labels)
# result.bilby_result.plot_corner()

# Option 2: Custom corner plot using the corner package
def plot_corner(flat_chain, labels, filename="corner_plot.png"):
    """Create a corner plot showing parameter correlations and confidence intervals."""
    fig = corner.corner(
        flat_chain,
        labels=labels,
        quantiles=[0.16, 0.5, 0.84],  
        show_titles=True,
        title_kwargs={"fontsize": 14},
        label_kwargs={"fontsize": 14},
        truths=np.median(flat_chain, axis=0),  # Show median values
        truth_color='red',
        smooth=1,
        bins=30,
        plot_datapoints=False,
        fill_contours=True,
        levels=[0.16, 0.5, 0.68],  
        color='k',
        alpha=0.5
    )
    for ax in fig.get_axes():
        ax.tick_params(axis='both', labelsize=12) 
    fig.savefig(filename, dpi=600, bbox_inches='tight')
    print(f"Corner plot saved to: {filename}")

# Flatten the samples for corner plot
# Note: For nested sampling, samples are weighted - this is a simplified version
flat_chain = result.samples.reshape(-1, result.samples.shape[-1])

# Create corner plot
plot_corner(flat_chain, result.latex_labels, filename="corner_plot.png")

# For a more accurate weighted corner plot, use bilby's built-in method:
# result.bilby_result.plot_corner(filename="corner_bilby.png")